# Visualization
This notebooks serves to visualize results of an individual run in detail.

In [ ]:
# core stuff
import gravann,os
import numpy as np
import pickle as pk
from tqdm import tqdm
import torch

# For animation etc.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import matplotlib.pyplot as plt
import imageio

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ",device)

# Define the run folder

In [ ]:
results_folder = "results/run_10_11_2020_ACC_siren/Churyumov-Gerasimenko.pk/LR=0.0001_loss=normalized_L1_loss_encoding=direct_encoding_batch_size=1000_target_sample=spherical_activation=Sigmoid/"
# results_folder = "results/run_10_11_2020_ACC_default/Churyumov-Gerasimenko.pk/LR=0.0001_loss=normalized_L1_loss_encoding=direct_encoding_batch_size=1000_target_sample=spherical_activation=Sigmoid/"

# Load run parameters and model

In [ ]:
#Load run parameters
with open(results_folder + "config.pk", "rb") as file:
    params = pk.load(file)
    sample = params["Sample"]
    encoding = getattr(gravann, params["Encoding"])()
    model = gravann.init_network(encoding,model_type=params["Model"])
    model.load_state_dict(torch.load(results_folder + "model.mdl"))

print(params)
with open("mascons/" + sample, "rb") as file:
    mascon_points, mascon_masses, name = pk.load(file)

    mascon_points = torch.tensor(mascon_points)
    mascon_masses = torch.tensor(mascon_masses)
    
c = gravann.compute_c_for_model(model,encoding, 
                                mascon_points, mascon_masses, 
                                use_acc = True if params["Type"]=="ACC" else False)
torch.cuda.empty_cache()

In [ ]:
gravann.plot_model_contours(model,encoding,c=c,heatmap=True);

In [ ]:
gravann.plot_model_vs_mascon_contours(model,encoding,mascon_points,mascon_masses,c=c,heatmap=True,levels = np.linspace(0,1.6, 16),mascon_alpha=0);

In [ ]:
fig, val = gravann.plot_model_mascon_acceleration("3dmeshes/"+sample,model,encoding,mascon_points,mascon_masses,plane="YZ", altitude=0.05,N=5000,c=c,logscale=True);

In [ ]:
levels =  np.linspace(2.35,2.4, 16)
mascon_alpha = 0
images = []
for offset in tqdm(np.linspace(-0.75,0.75,20)):
    fig = gravann.plot_model_vs_mascon_contours(model,encoding,mascon_points,mascon_masses,c=c,offset=offset,heatmap=True, levels = levels,mascon_alpha=mascon_alpha);
    fig.canvas.draw();
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)
    plt.close("all")

In [ ]:
gifPath = "gifs/contourf.gif"
imageio.mimsave(gifPath, images)

In [ ]:
# Display GIF in Jupyter, CoLab, IPython
with open(gifPath,'rb') as f:
    display.Image(data=f.read(), format='png')

https://github.com/matplotlib/matplotlib/issues/3919
https://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html#filled-contour-plots
https://docs.enthought.com/mayavi/mayavi/mlab.html